# KRR Final Project

This is the Final KRR mod. 2 Project from Mattia Moro (matr. 502259). The project consists in the creation of a music knowledge
base and it is divided in different sections:

- **SECTION 1** : Requirements, packaging installing and general setup;
- **SECTION 2** : Load and manage csv files;
- **SECTION 3** : Creation of RDF Graph;
- **SECTION 4** : Extension of Graph with RDFS and OWL;
- **SECTION 5** : SPARQL queries to the graph;
- **SECTION 6** : Integration with DBPedia.

## Section 1

In this section i will first install the required libraries : rdflib, sparqlwrapper, owlrl, gdown, and pandas. The first one is for creating the graph, the second for queriying it, the third is to add the ontology part to the graph and the last two are for 
download and manage the files from which the graph will be created


In [1]:
# requirements
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install rdflib sparqlwrapper owlrl gdown pandas
#pip install -q rdflib sparqlwrapper owlrl gdown pandas

Now we can just import such libraries, and in particular import from rdflib the classes to make graphs, prefixes

In [2]:
import gdown
import os
import pandas as pd
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
from rdflib import Namespace, Graph, URIRef, BNode, Literal, FOAF, RDF, RDFS, XSD, OWL
import owlrl
import re

## Section 2 ##

This section was given by the Professor Milanese, so it is the same as on the Colab file


In [3]:
if not os.path.exists("./data/"):
    os.mkdir("./data/")

In [4]:
# Download data
gdown.download(id="1UILSMenvqzFMsIhV6l4ZV1opJpRQZnwC", output="./data/rym_top_5000_all_time.csv", quiet=True)
gdown.download(id="1UNJ2iDX-xoWIekSA0ZwoqNOk3eAZjqrp", output="./data/albumlist.csv", quiet=True)

'./data/albumlist.csv'

In [5]:
df1 = pd.read_csv('data/albumlist.csv', encoding='mac_roman')
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [6]:
df2 = pd.read_csv('data/rym_top_5000_all_time.csv')
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379


In [7]:
df1.shape, df2.shape

((500, 6), (5000, 9))

In [8]:
df1[df1.Artist=="Eminem"]

,Number,Year,Album,Artist,Genre,Subgenre
243,244,2000,The Marshall Mathers LP,Eminem,Hip Hop,"Pop Rap, Hardcore Hip-Hop, Horrorcore"
274,275,1999,The Slim Shady LP,Eminem,Hip Hop,None


In [9]:
df2[df2["Artist Name"].str.contains(",")]

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
165,166.0,Igor,"Tyler, the Creator",17 May 2019,"Neo-Soul, Psychedelic Soul","breakup, LGBT, male vocals, love, concept albu...",3.91,"32,167",307
196,197.0,Flower Boy,"Tyler, the Creator",21 July 2017,West Coast Hip Hop,"introspective, lonely, LGBT, lush, warm, male ...",3.90,"29,598",194
265,266.0,ニンテンドーDS ポケモン ダイヤモンド&パール スーパーミュージックコレクション (Nin...,"増田順一 [Junichi Masuda], 一之瀬剛 [Go Ichinose] & 佐藤...",22 December 2006,"Video Game Music, Sequencer & Tracker","soothing, instrumental, triumphant, happy, upl...",4.03,790,5
416,417.0,NieR Gestalt & RepliCant,"岡部啓一 [Keiichi Okabe], 石濱翔 [Kakeru Ishihama], 帆...",21 April 2010,"Video Game Music, Neoclassical New Age","female vocals, epic, atmospheric, ethereal, lu...",3.91,937,5
429,430.0,Déjà vu,"Crosby, Stills, Nash & Young",11 March 1970,Folk Rock,"warm, melodic, male vocals, mellow, calm, rhyt...",3.91,"7,590",183
987,988.0,ゼルダの伝説 ～風のタクト～ (The Legend of Zelda: The Wind ...,"永田権太 [Kenta Nagata], 若井淑 [Hajime Wakai], 峰岸透 [...",19 March 2003,"Video Game Music, Sequencer & Tracker","fantasy, epic, instrumental, aquatic",3.88,561,4
1025,1026.0,"Crosby, Stills & Nash","Crosby, Stills & Nash",29 May 1969,Folk Rock,"love, pastoral, melodic, warm, male vocals, up...",3.83,"4,937",126
1125,1126.0,Sonny Side Up,"Dizzy Gillespie, Sonny Stitt & Sonny Rollins",January 1959,Bebop,"improvisation, instrumental, acoustic, male vo...",3.86,"1,335",14
1205,1206.0,Promises,"Floating Points, Pharoah Sanders & The London ...",26 March 2021,"Third Stream, Post-Minimalism","instrumental, minimalistic, meditative, atmosp...",3.79,"9,257",78
1427,1428.0,GBAポケモン ルビー&サファイア ミュージック・スーパーコンプリート (GBA Pokém...,"増田順一 [Junichi Masuda], 一之瀬剛 [Go Ichinose] & 青木...",26 April 2003,"Video Game Music, 16-bit","triumphant, playful, energetic, instrumental, ...",3.84,454,2


In [10]:
df1['a'] = df1.Album.str.lower()
df1.head()

,Number,Year,Album,Artist,Genre,Subgenre,a
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",sgt. pepper's lonely hearts club band
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",pet sounds
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",revolver
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",highway 61 revisited
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,rubber soul


In [11]:
df2['a'] = df2.Album.str.lower()
df2.head()

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews,a
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531,ok computer
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983,wish you were here
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870,in the court of the crimson king
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734,kid a
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379,to pimp a butterfly


In [12]:
merge = pd.merge(df1, df2, on="a")

In [13]:
merge.shape

(342, 16)

In [14]:
merge.head()

,Number,Year,Album_x,Artist,Genre,Subgenre,a,Ranking,Album_y,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",sgt. pepper's lonely hearts club band,24.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1 June 1967,"Psychedelic Pop, Pop Rock","psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock",pet sounds,20.0,Pet Sounds,The Beach Boys,16 May 1966,Baroque Pop,"Wall of Sound, warm, bittersweet, love, romant...",4.18,"36,305",727
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",revolver,11.0,Revolver,The Beatles,5 August 1966,"Pop Rock, Psychedelic Pop","psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock",highway 61 revisited,35.0,Highway 61 Revisited,Bob Dylan,30 August 1965,"Folk Rock, Singer/Songwriter","poetic, cryptic, surreal, sarcastic, male voca...",4.17,"26,338",604
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,rubber soul,73.0,Rubber Soul,The Beatles,3 December 1965,Pop Rock,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721


In [15]:
df1[df1.Artist == "The Beatles"]

,Number,Year,Album,Artist,Genre,Subgenre,a
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",sgt. pepper's lonely hearts club band
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",revolver
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,rubber soul
9,10,1968,"The Beatles (""The White Album"")",The Beatles,Rock,"Rock & Roll, Pop Rock, Psychedelic Rock, Exper...","the beatles (""the white album"")"
13,14,1969,Abbey Road,The Beatles,Rock,"Psychedelic Rock, Classic Rock, Pop Rock",abbey road
38,39,1963,Please Please Me,The Beatles,Rock,"Beat, Rock & Roll",please please me
52,53,1964,Meet The Beatles!,The Beatles,Rock,"Beat, Rock & Roll",meet the beatles!
306,307,1964,A Hard Day's Night,The Beatles,"Rock, Stage & Screen","Soundtrack, Beat, Pop Rock",a hard day's night
330,331,1965,Help!,The Beatles,"Rock, Stage & Screen","Beat, Soundtrack, Pop Rock",help!
391,392,1970,Let It Be,The Beatles,Rock,Pop Rock,let it be


In [16]:
df2[df2["Artist Name"] == "The Beatles"]

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews,a
7,8.0,Abbey Road,The Beatles,26 September 1969,Pop Rock,"melodic, warm, male vocals, bittersweet, summe...",4.25,"44,544",961,abbey road
10,11.0,Revolver,The Beatles,5 August 1966,"Pop Rock, Psychedelic Pop","psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160,revolver
23,24.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1 June 1967,"Psychedelic Pop, Pop Rock","psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863,sgt. pepper's lonely hearts club band
30,31.0,The Beatles [White Album],The Beatles,22 November 1968,"Pop Rock, Rock","eclectic, melodic, playful, male vocals, quirk...",4.14,"35,545",1085,the beatles [white album]
72,73.0,Rubber Soul,The Beatles,3 December 1965,Pop Rock,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721,rubber soul
78,79.0,Magical Mystery Tour,The Beatles,27 November 1967,"Psychedelic Pop, Pop Rock","psychedelic, melodic, playful, surreal, male v...",4.06,"27,318",560,magical mystery tour
1894,1895.0,Help!,The Beatles,6 August 1965,Pop Rock,"bittersweet, melodic, love, male vocals, roman...",3.67,"20,697",380,help!
1957,1958.0,Love,The Beatles,26 November 2006,"Pop Rock, Mashup, Psychedelic Rock","melodic, male vocals, mashup, psychedelic, hyp...",3.75,"2,309",142,love
2155,2156.0,A Hard Day's Night,The Beatles,10 July 1964,"Merseybeat, Pop Rock","love, romantic, melodic, male vocals, energeti...",3.65,"20,388",462,a hard day's night
4818,4819.0,Let It Be,The Beatles,18 May 1970,Pop Rock,"male vocals, melodic, warm, bittersweet, energ...",3.55,"21,363",414,let it be


In [17]:
merge[merge["Artist"] == "The Beatles"]

,Number,Year,Album_x,Artist,Genre,Subgenre,a,Ranking,Album_y,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock",sgt. pepper's lonely hearts club band,24.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1 June 1967,"Psychedelic Pop, Pop Rock","psychedelic, playful, melodic, male vocals, wa...",4.13,"43,576",863
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock",revolver,11.0,Revolver,The Beatles,5 August 1966,"Pop Rock, Psychedelic Pop","psychedelic, melodic, male vocals, drugs, ecle...",4.23,"43,178",1160
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock,rubber soul,73.0,Rubber Soul,The Beatles,3 December 1965,Pop Rock,"melodic, love, male vocals, bittersweet, roman...",4.03,"33,508",721
11,14,1969,Abbey Road,The Beatles,Rock,"Psychedelic Rock, Classic Rock, Pop Rock",abbey road,8.0,Abbey Road,The Beatles,26 September 1969,Pop Rock,"melodic, warm, male vocals, bittersweet, summe...",4.25,"44,544",961
170,392,1970,Let It Be,The Beatles,Rock,Pop Rock,let it be,365.0,Let It Be,The Replacements,September 1984,Alternative Rock,"passionate, melodic, energetic, bittersweet, m...",3.91,"10,775",222
171,392,1970,Let It Be,The Beatles,Rock,Pop Rock,let it be,4819.0,Let It Be,The Beatles,18 May 1970,Pop Rock,"male vocals, melodic, warm, bittersweet, energ...",3.55,"21,363",414
223,307,1964,A Hard Day's Night,The Beatles,"Rock, Stage & Screen","Soundtrack, Beat, Pop Rock",a hard day's night,2156.0,A Hard Day's Night,The Beatles,10 July 1964,"Merseybeat, Pop Rock","love, romantic, melodic, male vocals, energeti...",3.65,"20,388",462
243,331,1965,Help!,The Beatles,"Rock, Stage & Screen","Beat, Soundtrack, Pop Rock",help!,1895.0,Help!,The Beatles,6 August 1965,Pop Rock,"bittersweet, melodic, love, male vocals, roman...",3.67,"20,697",380


## Section 3 : graph creation ##

In this section i will create the graph.
First of all, i declared the used prefixes. To simplify my work, before starting i looked on DBPedia how the "musical knowledge" was modelled, and i had tried to keep the same predicates/name of classes (not for plagiarism, but for conceptually be more next to DBPedia for the last part) and also prefixes.

Then i defined two lists with forbidden characters in RDF syntax (will be useful later). Notice that there was one album with a weird unicode space, so to the second list i added the charachter correspondent to its code.

### Triples creation ###

Now, by iterating over the rows of the dataframe, i had access to the single rows.
The first thing that i have done to all cells of the dataframe was to remove the forbidden characters and replace them with underscores or simply remove them (only & was replaced by "e").
This was done in order to create correct resources from the syntax point of view.
For some of them other operation were needed (for example if the first character was a number the uri cannot be created, so i put an _ in that case).

From the Album resource (that was the main one) i created:

- label : i was careful to keep the same criteria of DBPedia for this one (Simply first letter of any word in capital);
- release year : also here i used the same DBPedia predicate, then the year as integer;
- release date : the date was not in the correct format, so i replaced all month names with their numbers, split the date and then rearrange it in such a way that the year was at the first place (this is the xsd standard);
- for the dates without day i created another predicate, this time i didn't replace the month name with the number, because i cannot use the xsd^^date format;
- descriptors : every album has its own descriptors as separate objects, so that from one descriptors in the future will be possible to ask for correspondent albums;
- Then there was a part for all numbers (rankings, rating, number of ratings) saved as integers or floats;
- artist: also for this resource i kept the same DBPedia labels, then i add with the predicate "artist" the triple "dbr:album dbo:artist dbr:artist";
- The last part is dedicated to genres, here i added as genres the ones from the columns "Genre" , "Genres" and "subgenres", as triples like "album has genre/subgenre name genre/subgenre".

There are some missing informations that will be inferred later (types and in general inferences). Albums and Artists were declares as individuals.


In [18]:
#at first, define the prefixes.
ex = Namespace('http://example.org/')
rdf = Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
dbr = Namespace('http://dbpedia.org/resource/')
dbp = Namespace('http://dbpedia.org/property/')
dbo = Namespace('http://dbpedia.org/ontology/')
#now i have created the graph and two lists with forbidden characters in IRIs, so that they can be replaced in the creation of the triples, obviously, these characters are included in the labels, except for " and '.
gdef = Graph()
triples = []
prob = [":","!",",","-","(",")","?",'"',"#","/"]
und = [" ","'",".",chr(0xa0)]
#i found a cell with a space that is different from the traditional one (unicode 0020), that is unicode 00A0 (no break space), so i added this special space to the list of special characters
for index,row in merge.iterrows():
  prova = str(row['a'])
  artistt = str(row['Artist'])
  for car in str(row['a']):
    if car in prob:
      prova = str(prova).replace(car,"")
    else:
      if car in und:
        prova = str(prova).replace(car,"_")
      else:
        if car == '&':
          prova = str(prova).replace(car,"e")
  if prova[0].isdecimal() == True:
    prova = "_"+str(prova)
  etichetta = str(row['a']).replace("'"," ").replace('"',' ')
  if prova.isdecimal() == False:
        if len(prova.split("_")) == 1:
            prova = prova.capitalize()
        else:
            provviz = prova.split("_")
            for i in range(len(provviz)):
                provviz[i] = provviz[i].capitalize()
            prova = "_".join(provviz)
  if etichetta.isdecimal() == False:
        if len(etichetta.split(" ")) == 1:
            etichetta = etichetta.capitalize()
        else:
            provvi = etichetta.split()
            for i in range(len(provvi)):
                provvi[i] = provvi[i].capitalize()
            etichetta = " ".join(provvi)
  #album label and individual declaration         
  lab = 'dbr.'+str(prova)+', RDFS.label, Literal('+'"'+str(etichetta)+'", lang="en")'
  labt = eval(lab)
  triples.append(labt)
  ind = 'dbr.'+str(prova)+', RDF.type, OWL.NamedIndividual'
  indtup = eval(ind)
  triples.append(indtup)
  #release year
  yea = 'dbr.'+str(prova)+', dbp.relyear, Literal("'+str(row['Year'])+'", datatype=XSD.integer)'
  yeatup = eval(yea)
  triples.append(yeatup)
  if len(str(row['Release Date']).split(" ")) == 3:
    schifo = str(row['Release Date']).replace(" ","-").replace("January","01").replace("February","02").replace("March","03").replace("April","04").replace("May","05").replace("June","06").replace("July","07").replace("August","08").replace("September","09").replace("October","10").replace("November","11").replace("December","12")
    if len(schifo) < 10:
      schifo = "0"+schifo
    listas = schifo.split("-")
    #release complete date
    data = "dbr."+str(prova)+", dbp.released, Literal('"+str(listas[2])+'-'+str(listas[1])+'-'+str(listas[0])+"' ,datatype=XSD.date)"
    datatup = eval(data)
    triples.append(datatup)
  else:
    if len(str(row['Release Date']).split(" ")) == 2:
      #month year date
      datamonca = "dbr."+str(prova)+", dbp.month_year, Literal('"+str(row['Release Date'])+"')"
      datamoncatup = eval(datamonca)
      triples.append(datamoncatup)
  deslist = str(row['Descriptors']).split(", ")
  for des in deslist:
    for let in des:
      if let in prob:
        des = des.replace(let,"")
      else:
        if let in und:
          des = des.replace(let,"_")
        else:
          if let == '&':
            des = des.replace(let,"e")
    #descriptors for album
    tripdee = "dbr."+str(prova)+", dbp.has_descriptor, dbr."+str(des)
    tripdtup = eval(tripdee)
    triples.append(tripdtup)
  #rank, average rating, number of ratings and number of views
  triprank = 'dbr.'+str(prova)+', dbp.rank, Literal("'+str(row['Ranking'])+'" ,datatype=XSD.float)'
  tripranktup = eval(triprank)
  triples.append(tripranktup)
  tripravrat = 'dbr.'+str(prova)+', dbp.average_rating, Literal("'+str(row['Average Rating'])+'", datatype=XSD.float)'
  tripavratup = eval(tripravrat)
  triples.append(tripavratup)
  tripnof = 'dbr.'+str(prova)+', dbp.number_of_ratings, Literal("'+str(row['Number of Ratings']).replace(",","")+'", datatype=XSD.integer)'
  tripnoftup = eval(tripnof)
  triples.append(tripnoftup)
  tripnor = 'dbr.'+str(prova)+', dbp.number_of_reviews, Literal("'+str(row['Number of Reviews']).replace(",","")+'", datatype=XSD.integer)'
  tripnortup = eval(tripnor)
  triples.append(tripnortup)
  for car2 in str(row['Artist']):
    if car2 in prob:
      artistt = str(artistt).replace(car2,"")
    else:
      if car2 in und:
        artistt = str(artistt).replace(car2,"_")
      else:
        if car2 == '&':
          artistt = str(artistt).replace(car2,"e")
  if artistt[0].isdecimal() == True:
    artistt = "_"+str(artistt)
  #artist label
  lab2 = 'dbr.'+str(artistt)+', RDFS.label, Literal('+'"'+str(row['Artist']).replace("'"," ").replace('"',' ')+'", lang="en")'
  labt2 = eval(lab2)
  triples.append(labt2)
  #triple album made by artist/ artist as individuals
  artalb = "dbr."+str(prova)+", dbo.artist, dbr."+str(artistt)
  tupartalb = eval(artalb)
  triples.append(tupartalb)
  inda = 'dbr.'+str(artistt)+', RDF.type, OWL.NamedIndividual'
  indatup = eval(inda)
  triples.append(indatup)
  subg = str(row['Subgenre'])
  subglist = subg.split(", ")
  contlist = 0
  contword = 0
  for car2 in subglist:
    for c in car2:
      if c in prob:
        subglist[contlist] = str(subglist[contlist]).replace(c,"")
      else:
        if c in und:
          subglist[contlist] = str(subglist[contlist]).replace(c,"_")
        else:
          if c == '&':
            subglist[contlist] = str(subglist[contlist]).replace("&","e")
    contlist += 1
  contlist = 0
  genre = str(row['Genre'])
  genrelist = genre.split(", ")
  for car2 in genrelist:
    for c in car2:
      if c in prob:
        genrelist[contlist] = str(genrelist[contlist]).replace(c,"")
      else:
        if c in und:
          genrelist[contlist] = str(genrelist[contlist]).replace(c,"_")
        else:
          if c == '&':
            genrelist[contlist] = str(genrelist[contlist]).replace("&","e")
    contlist += 1
  contlist = 0
  genres = str(row['Genres'])
  genreslist = genres.split(", ")
  for car2 in genreslist:
    for c in car2:
      if c in prob:
        genreslist[contlist] = str(genreslist[contlist]).replace(c,"")
      else:
        if c in und:
          genreslist[contlist] = str(genreslist[contlist]).replace(c,"_")
        else:
          if c == '&':
            genreslist[contlist] = str(genreslist[contlist]).replace("&","e")
    contlist += 1
  contlist = 0
  for genre in genreslist:
    #album has genre genre
    trips = 'dbr.'+str(prova)+', dbo.genre, dbr.'+str(genre)
    triptups = eval(trips)
    triples.append(triptups)
  for genres in genreslist:
    #here is the same as above, but there were two colums with genres, so i added as genres also these ones
    trippz = 'dbr.'+str(prova)+', dbo.genre, dbr.'+str(genres)
    tripztups = eval(trippz)
    triples.append(tripztups) 

  if subglist[0] != 'None':
    for subgs in subglist:
      #album has subgenre subgenre
      playsub = 'dbr.'+str(prova)+', dbo.sub_genre, dbr.'+str(subgs)
      playsubtup = eval(playsub)
      triples.append(playsubtup)


## Section 4 : rdfs and owl##

Here there is the rdfs/owl part.
In particular, i created two taxonomies, by putting music_artists as subclass of person and album as subclass of music work. All other classes are then declared as owl:Class.
Then i had declared the object properties with their ranges and domains, saying also that they are all asymmetric and irreflexive (they are not functional or inverse functional), in this part there is also a taxonomy for object properties (subgenres and genres).
At last, i put all predicates that have a literal as object as datatypeproperty.

In [19]:
# class and taxonomy and disjunction
m = 'dbr.music_artist, RDF.type, OWL.Class'
mtup = eval(m)
triples.append(mtup)
p = 'dbr.person, RDF.type, OWL.Class'
ptup = eval(p)
triples.append(ptup)
sub1 = 'dbr.music_artist, RDFS.subClassOf, dbr.person'
sub1tup = eval(sub1)
triples.append(sub1tup)
mg = 'dbr.album, RDF.type, OWL.Class'
mgtup = eval(mg)
triples.append(mgtup)
mw = 'dbr.music_work, RDF.type, OWL.Class'
mwtup = eval(mw)
triples.append(mwtup)
sub2 = 'dbr.album, RDFS.subClassOf, dbr.music_work'
sub2tup = eval(sub2)
triples.append(sub2tup)
d = 'dbr.descriptor, RDF.type, OWL.Class'
dtup = eval(d)
triples.append(dtup)
ms = 'dbr.music_subgenre, RDF.type, OWL.Class'
mstup = eval(ms)
triples.append(mstup)
mg = 'dbr.music_genre, RDF.type, OWL.Class'
mgtup = eval(mg)
triples.append(mgtup)
sub3 = 'dbr.music_subgenre, RDFS.subClassOf, dbr.music_genre'
sub3tup = eval(sub3)
triples.append(sub3tup)
dis = 'dbr.person, OWL.disjointWith, dbr.music_work'
distup = eval(dis)
triples.append(distup)
#properties: dataype properties for the ones which objects are just integer/floats/dates
rly ='dbp.relyear, RDF.type,  OWL.DatatypeProperty'
rlytup = eval(rly)
triples.append(rlytup)
rl = 'dbp.released, RDF.type, OWL.DatatypeProperty'
rltup = eval(rl)
triples.append(rltup)
my = 'dbp.month_year, RDF.type, OWL.DatatypeProperty'
mytup = eval(my)
triples.append(mytup)
#as for the one below, for all object properties domain and range were declared, but also the inverse properties and the fact that
#they are asymmetric and irreflexive. 
hd = 'dbp.has_descriptor, RDF.type, OWL.ObjectProperty'
hdtup = eval(hd)
triples.append(hdtup)
hdirr = 'dbp.has_descriptor, RDF.type, OWL.IrreflexiveProperty'
hdirrtup = eval(hdirr)
triples.append(hdirrtup)
has = 'dbp.has_descriptor, RDF.type, OWL.AsymmetricProperty'
hastup = eval(has)
triples.append(hastup)
dinv = 'dbp.describes, RDF.type, OWL.ObjectProperty'
dinvtup = eval(dinv)
triples.append(dinvtup)
dinvdef = 'dbp.describes, OWL.inverseOf, dbp.has_descriptor'
dinvdeftup = eval(dinvdef)
triples.append(dinvdeftup)
hdd = 'dbp.has_descriptor, RDFS.domain, dbr.album'
hddtup = eval(hdd)
triples.append(hddtup)
hdr = 'dbp.has_descriptor, RDFS.range, dbr.descriptor'
hdrtup = eval(hdr)
triples.append(hdrtup)
rk = 'dbp.rank, RDF.type, OWL.DatatypeProperty'
rktup = eval(rk)
triples.append(rktup)
avr = 'dbp.average_rating, RDF.type, OWL.DatatypeProperty'
avrtup = eval(avr)
triples.append(avrtup)
nof = 'dbp.number_of_ratings, RDF.type, OWL.DatatypeProperty'
noftup = eval(nof)
triples.append(noftup)
nor = 'dbp.number_of_reviews, RDF.type, OWL.DatatypeProperty'
nortup = eval(nor)
triples.append(nortup)
arr = 'dbo.artist, RDF.type, OWL.ObjectProperty'
arrtup = eval(arr)
triples.append(arrtup)
arirr = 'dbo.artist, RDF.type, OWL.IrreflexiveProperty'
arirrtup = eval(arirr)
triples.append(arirrtup)
aras = 'dbo.artist, RDF.type, OWL.AsymmetricProperty'
arastup = eval(aras)
triples.append(arastup)
arrinv = 'dbo.made, RDF.type, OWL.ObjectProperty'
arrinvtup = eval(arrinv)
triples.append(arrinvtup)
arrinvdef = 'dbo.made, OWL.inverseOf, dbo.artist'
arrinvdeftup = eval(arrinvdef)
triples.append(arrinvdeftup)
arrd = 'dbo.artist, RDFS.domain, dbr.album'
arrdtup = eval(arrd)
triples.append(arrdtup)
arra = 'dbo.artist, RDFS.range, dbr.music_artist'
arratup = eval(arra)
triples.append(arratup)
sgen = 'dbo.sub_genre, RDF.type, OWL.ObjectProperty'
sgentup = eval(sgen)
triples.append(sgentup)
sgirr = 'dbo.sub_genre, RDF.type, OWL.IrreflexiveProperty'
sgirrtup = eval(sgirr)
triples.append(sgirrtup)
sgas = 'dbo.sub_genre, RDF.type, OWL.AsymmetricProperty'
sgastup = eval(sgas)
triples.append(sgastup)
sgend = 'dbo.sub_genre, RDFS.domain, dbr.album'
sgendtup = eval(sgend)
triples.append(sgendtup)
sgenr = 'dbo.sub_genre, RDFS.range, dbr.music_subgenre'
sgenrtup = eval(sgenr)
triples.append(sgenrtup)
gen = 'dbo.genre, RDF.type, OWL.ObjectProperty'
gentup = eval(gen)
triples.append(gentup)
girr = 'dbo.genre, RDF.type, OWL.IrreflexiveProperty'
girrtup = eval(girr)
triples.append(girrtup)
gas = 'dbo.genre, RDF.type, OWL.AsymmetricProperty'
gastup = eval(gas)
triples.append(gastup)
gend = 'dbo.genre, RDFS.domain, dbr.album'
gendtup = eval(gend)
triples.append(gendtup)
genr = 'dbo.genre, RDFS.range, dbr.music_genre'
genrtup = eval(genr)
triples.append(genrtup)
gtx = 'dbo.sub_genre, RDFS.subPropertyOf, dbo.genre'
gtxtup = eval(gtx)
triples.append(gtxtup)
subinv = 'dbo.subgenre_of, RDF.type, OWL.ObjectProperty'
subinvtup = eval(subinv)
triples.append(subinvtup)
subinvdef = 'dbo.subgenre_of, OWL.inverseOf, dbo.sub_genre'
subinvdeftup = eval(subinvdef)
triples.append(subinvdeftup)
geninv = 'dbo.genre_of, RDF.type, OWL.ObjectProperty'
geninvtup = eval(geninv)
triples.append(geninvtup)
geninvdef = 'dbo.genre_of, OWL.inverseOf, dbo.genre'
geninvdeftup = eval(geninvdef)
triples.append(geninvdeftup)
#taxonomy between properties
invtax = 'dbo.subgenre_of, RDFS.subPropertyOf, dbo.genre_of'
invtaxtup = eval(invtax)
triples.append(invtaxtup)
#triplesadded to graph, printed later
for triple in triples:
  gdef.add(triple)
gdef.bind("dbr",dbr)
gdef.bind("dbp",dbp)
gdef.bind("dbo",dbo)


In [20]:
print(gdef.serialize())
print(len(gdef))

@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbp: <http://dbpedia.org/property/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

dbo:made a owl:ObjectProperty ;
    owl:inverseOf dbo:artist .

dbo:subgenre_of a owl:ObjectProperty ;
    rdfs:subPropertyOf dbo:genre_of ;
    owl:inverseOf dbo:sub_genre .

dbp:average_rating a owl:DatatypeProperty .

dbp:describes a owl:ObjectProperty ;
    owl:inverseOf dbp:has_descriptor .

dbp:month_year a owl:DatatypeProperty .

dbp:number_of_ratings a owl:DatatypeProperty .

dbp:number_of_reviews a owl:DatatypeProperty .

dbp:rank a owl:DatatypeProperty .

dbp:released a owl:DatatypeProperty .

dbp:relyear a owl:DatatypeProperty .

dbr:A_Hard_Day_S_Night a owl:NamedIndividual ;
    rdfs:label "A Hard Day S Night"@en ;
    dbo:artist dbr:The_Beatles ;
    dbo:genre dbr:Merseybeat,
        

## SECTION 5: SPARQL Queries ###

Now that the graph is done, here are some sparql queries done with sparqlwrapper

In [21]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.addDefaultGraph("ex:502259MM")
sparql.method = 'POST'
for s,p,o in gdef:
    sparql.setQuery(f"INSERT DATA {{{s.n3()} {p.n3()} {o.n3()}.}}")
    sparql.query()

URLError: <urlopen error [WinError 10061] Impossibile stabilire la connessione. Rifiuto persistente del computer di destinazione>

Now that we have set the endpoint and fill the graph with my immatriculation number, we can definitely query it.
With a graph modelled like this, we can do several queries, and i'll put some examples below.

### Query 1 ###

This graph in "centered" on the "album" column of the csv, so the first query could be simply a list of all these albums.
I didn't explicitly put the type to the instances of this class, because it can be returned later, but notice that we can still obtain the list of all albums asking for the first term of the triple "album artist artist":


In [ ]:
sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/> 
                   PREFIX dbp: <http://dbpedia.org/property/> 
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y  where { ?s dbo:artist ?x. ?s rdfs:label ?y}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

Here we can see that the variable "y" has been substituted with the album label (that is better presented than the album URI, identified here with variablee "s"). By changing the place of the variable, it is possible to do this also for artists's names.

### Query 2 ###

Another interesting thing that we can do is to try to build a sort of "rank" between albums. In fact, notice that we have different indicators thanks to which we can build different rankings.
For example, this is the ranking for (average) rating:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y  ?x where { ?s dbp:average_rating ?x . ?s rdfs:label ?y } order by desc(?x)""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

As we can see, the most high-ranked album is "A love supreme". This can be done, by changing the predicate, also with the number of reviews and the number of ratings.
Another thing that could be useful is to see the most recent/old album, or to search for all album publisced in a year or in a decade, here some examples: 

### Query 3 : year queries ###

Oldest Albums:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y  ?x where { ?s dbp:released ?x . ?s rdfs:label ?y } order by ?x""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

(by adding "desc" on the order we get the most recent).

All albums released in 2002:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y where { ?s dbp:relyear "2002"^^xsd:int . ?s rdfs:label ?y }""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

All ablums from the 80s:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y  ?z where { ?s dbp:relyear ?z . ?s rdfs:label ?y FILTER ((?z >= 1980) && (?z <= 1990))}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

Is "Master of Puppets" made by Metallica published in the 80s ? (Actually yes, i know, but let's see what SPARQL says)

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   ask where { dbr:Master_Of_Puppets dbp:relyear ?z FILTER ((?z >= 1980) && (?z <= 1990))}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results)

### Query 4 ###

Another useful thing from the graph are the descriptors, so we can simply query for the list of descriptors of a specific album or retrive all albums that have a specific descriptor:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y ?z ?w where { {?x dbp:has_descriptor ?z. ?x rdfs:label ?y} UNION {?x dbp:has_descriptor dbr:psychedelic. ?x rdfs:label ?w }}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

### Query 5 ###

There is also a list of artists, so we can, for example, retrive all albums done by the beatles:

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y ?z where { ?x dbo:artist dbr:The_Beatles. ?x rdfs:label ?y. dbr:The_Beatles rdfs:label ?z }""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

### Query 6 ###

We can also check for the subgenres of an album (genres will be inferred) or for all albums with a specific subgenre:


In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select ?y ?z where { {?x dbo:sub_genre dbr:Country_Rock. ?x rdfs:label ?y} UNION {dbr:Master_Of_Reality dbp:sub_genre ?z} }""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

## Materializing inferences ##

We can also do that with the closure functions from owlrl, but also with the SPARQL INSERT method. The Inferences will be grouped in base on the rdfs or owl rule that they involve.

### Rdfs 7 ###

This rule says that if a subjec and an object are linked from a generic property p that is subproperty of another property q, then the subject and the object share also the q property

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   insert {?w ?q ?z} where { ?w ?p ?z. ?p rdfs:subPropertyOf ?q . } """)
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   select * where {?s dbo:genre ?o}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

### Rdfs2 - rdfs3 ##

These rules are the one of "domain" and "range". Applied to an object property they allow us to set the class of the subject (domain) and of the object (range). In this graph, we can use them on: 
- has_descriptor from album to descriptor
- sub_genre from album to subgenre
- artist from album to artist
- genre from album to genre

In the following cell, via "insert {} where" i will set the template and the rules to build it, then add to the graph. This will be done automatically for all the object properties.


In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   insert {?w rdf:type ?y . ?x rdf:type ?z} where { ?w ?b ?x. ?b a owl:ObjectProperty ; rdfs:domain ?y ; rdfs:range ?z . } """)
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   select * where {?s rdf:type ?o}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

### Rdfs 9 ###

This rule says that if an individual is an instance of a class and that class is subclass of another class, then the individual is an instance also of the mother class 

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   insert {?w rdf:type ?y} where { ?w a ?x. ?x rdfs:subClassOf ?y } """)
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   select * where {?s rdf:type ?o}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

## prp inv 1/2 ##

This rule is the one that inverts instances when there is an object property that is the inverse of another one.

In [ ]:

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   PREFIX dbo: <http://dbpedia.org/ontology/>
                   insert {?x ?p ?w.} where { ?w ?q ?x. ?p owl:inverseOf ?q.} """)
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""PREFIX dbr: <http://dbpedia.org/resource/>
                   PREFIX dbp: <http://dbpedia.org/property/>
                   select * where {?s ?p ?o}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

These are the main rules, the remaining ones were not included in the graph or weren't as "relevant" as these ones.

## SECTION 6: Integration with DBPedia ##

In this part the goal is to add datas to the graph by taking them from DBPedia, the way to do it is with federated queries.
This special type of query allow us to query two graphs at the same time simply by indicating their URLs. 
Unfortunately, if the above graph ex:502259 is kept as defaul one the queries don't work. This means that we need to do federated queries without a default graph, but putting inside them the links from both DBPedia and our endpoint.
But the first thing to do is to open gitbash, execute virtuoso, go to ./virtuoso.sh isql and write:

- GRANT INSERT ON "DB.DBA.SPARQL_SINV_2" TO "SPARQL";
- GRANT SELECT ON "DB.DBA.SPARQL_SINV_2" TO "SPARQL";
- GRANT EXECUTE ON "DB.DBA.SPARQL_SINV_2" TO "SPARQL";

to get permissions of access to dbpedia.
The biggest issue in this part is that (for some reasons unknown by me) the queries didn't work by putting in them common variables between the two graphs (even thought i checked that my labels and the ones from DBPedia were exactly the same), so the following ones are just a "showcase". They show the mechanism to do such queries for all URIs but i had to specify one to make them work.


The proof it's just this first query. Originally i asked for URIs with the same label using a variable, but the result is an empty table. This is inexplicable, in fact if i run the query for single artists it actually works (as i have done in this case). In reality, for how i had modelled the Graph, i took care of the fact that my resources were equal to the ones in DBPedia to facilitate the integration (actually most of them are the same, in fact their URIs link to the DBPedia correspontant page). The uniques one that do not match are the ones for which DBPedia introsuced a disambiguation in the name (e.g. "Nirvana (band)"). For this reason, i didn't expect that this query returned all the artists, but neither that returns an empty table, mainly because i manually checked a lot of labels.

In [ ]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.method = 'POST'
sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM> { ?x owl:sameAs ?y} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Metallica"@en } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?y rdfs:label "Metallica"@en. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM> { ?x owl:sameAs ?y} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Metallica"@en } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?y rdfs:label "Metallica"@en. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

The same reasoning is done for the albums. In fact my labels (also here except for disambiguations) have the same format of the ones from DBPedia, and also the URIs. But the "entire" query continues to be empty. So, also here i took one album for the query, making them match throught the label.

In [ ]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.method = 'POST'
sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM> { ?x owl:sameAs ?y} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Master Of Reality"@en } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?y rdfs:label "Master Of Reality"@en. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM> { ?x owl:sameAs ?y} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Master Of Reality"@en } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?y rdfs:label "Master Of Reality"@en. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

This Query it is done to identify which ones are bands from the music artists. I checked for the exsistence of the predicate "dbo:bandMember" on DBPedia, existing obviously only for the bands, and with a construct query i associated artists with a label in my graph (for the reasons said above, in this case i specified a band name) with artists with the same label on dbpedia that had this predicate. Then i "constructed" that such artist are in the "music_band" class, i added all members and said that are music artist.
At last, after a modification of the result, i added these triples to the original graph.

In [ ]:

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM> { ?x a dbr:music_band; dbo:bandMember ?f. ?f a dbr:music_artist.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Metallica"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Metallica"@en ;
        dbo:bandMember ?f }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
DELETE { GRAPH <ex:502259MM> { ?x a dbr:person.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_band; rdfs:label "Metallica"@en. } 

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM> { ?x a dbr:music_band; dbo:bandMember ?f. ?f a dbr:music_artist.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Metallica"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Metallica"@en ;
        dbo:bandMember ?f }

} LIMIT 10""")
 
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
m = results
print(m.decode())

With the same criteria, i added the death artists:

In [ ]:

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM>  { ?x dbo:deathDate ?f .} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "John Lennon"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "John Lennon"@en ;
        dbo:deathDate ?f }
} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM>  { ?x dbo:deathDate ?f .} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "John Lennon"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "John Lennon"@en ;
        dbo:deathDate ?f }
} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
c = results.decode()
print(c)

In this case, the purpose is to add other works from the artists in the graph (on dbpedia it is viewable on the section "is dbo:artist of" present for most of the artists).

In [ ]:

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM>  {?y dbo:artist ?x.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Black Sabbath"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Black Sabbath"@en .
    ?y dbo:artist ?x }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()

sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT{ GRAPH <ex:502259MM>  {?y dbo:artist ?x.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:music_artist; rdfs:label "Black Sabbath"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Black Sabbath"@en .
    ?y dbo:artist ?x }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
f = results.decode()
print(f)

Now retrive the album producer:

In [ ]:
sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM>  {?x dbp:producer ?y.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Nevermind"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Nevermind"@en ; dbp:producer ?y. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM>  {?x dbp:producer ?y.} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Nevermind"@en. } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Nevermind"@en ; dbp:producer ?y. }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

Now, by taking the genre of an ablum, we will retrive all artists that play that same genre:

In [ ]:
sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM>  {?w dbp:plays_same ?z} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Nevermind"@en; dbo:artist ?w } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Nevermind"@en ; dbp:genre ?y. ?y dbp:artist ?z }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM>  {?w dbp:plays_same ?z} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Nevermind"@en; dbo:artist ?w } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Nevermind"@en ; dbp:genre ?y. ?y dbp:artist ?z }

} LIMIT 10""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())

Now, as last query, let's see that if one member of the band that made an album had made other albums after that one.
To do such query i will ask for the release years of all works and then compare them with a filter:

In [ ]:
sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
INSERT { GRAPH <ex:502259MM>  {?z dbo:made ?f. ?f dbp:relyear ?g} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Abbey Road"@en; dbp:relyear ?w  } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Abbey Road"@en ; dbp:artist ?y. ?y dbo:formerBandMember ?z. ?f dbo:artist ?z ; dbp:relyear ?g }
FILTER ( ?g > ?w )
}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()


sparql.setQuery("""
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT { GRAPH <ex:502259MM>  {?z dbo:made ?f. ?f dbp:relyear ?g} }
WHERE
{ 
  SERVICE <http://localhost:8890/sparql/> { 
    ?x a dbr:album; rdfs:label "Abbey Road"@en; dbp:relyear ?w  } 
  SERVICE <http://dbpedia.org/sparql> { 
    ?x rdfs:label "Abbey Road"@en ; dbp:artist ?y. ?y dbo:formerBandMember ?z. ?f dbo:artist ?z ; dbp:relyear ?g }
FILTER ( ?g > ?w )
}""")
sparql.setReturnFormat(TURTLE)
results = sparql.queryAndConvert()
print(results.decode())